In [32]:
import os
import stat
import time
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document, StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from colorama import Fore
import warnings

warnings.filterwarnings("ignore")

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGUAGE_MODEL = "gpt-3.5-turbo-instruct"
LANGUAGE_MODEL = "gpt-4-turbo"


In [33]:
template: str = """/
    فرض کن تو یک مفسر مثنوی هستی. جواب سوال زیر را بده: /
      {question} /
   از محتوای زیر برای پیدا کردن مفاهیم و زمینه مربوطه استفاده کن. محتوای زیر از جلسات تفسیر مثنوی معنوی عبدالکریم سروش گرفته شده است./
      {context} /
      این محتوا از جلسه شماره زیر گرفته شده است:/
      {doc_name}/
       . سعی کن از این محتوا برای فهمیدن داستان و تاریخ مربوطه و ابیات مجاور بیت مورد سوال استفاده کنی./
       در پاسخی که میدهی سعی کن به زمینه داستانی و تاریخی و اشعار مجاور شعر مورد سوال در مثنوی اشاره کنی./
       در پاسخ خود به شماره جلسه ای که در مورد این شعر صحبت شده نیز اشاره کن./
       در پاسخ خود به تمام جزییاتی که از متن جلسه دریافت می کنی و مرتبط با سوال مطرح شده است اشاره کن.
    """

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question", "context"],
    template="{question}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

model = ChatOpenAI()

In [34]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


def load_documents():
    """
    Load all text files from a directory, split them into chunks,
    and add metadata with 'doc_id' and 'chunk_index' for each chunk.
    """
    loader = DirectoryLoader("./output_text/", glob="*.txt")  # Load all .txt files
    raw_documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=0,
        separators=["\n\n", "\n", " ", ""]
    )
    
    all_chunks = []
    for raw_doc in raw_documents:
        # Get a document identifier. Here we use the 'source' metadata if available.
        doc_id = raw_doc.metadata.get("source", "unknown")
        chunks = text_splitter.split_text(raw_doc.page_content)
        for idx, chunk in enumerate(chunks):
            new_doc = Document(page_content=chunk, metadata={"doc_id": doc_id, "chunk_index": idx})
            all_chunks.append(new_doc)
    return all_chunks


def load_embeddings(documents, user_query):
    """
    Create or load a Chroma vector store from a set of documents.
    """
    persist_directory = './chroma_cache'  # Directory to store embeddings
    embedding_model = OpenAIEmbeddings()

    # Ensure the directory exists and has write permissions
    if not os.path.exists(persist_directory):
        os.makedirs(persist_directory, exist_ok=True)
    else:
        if not os.access(persist_directory, os.W_OK):
            print(f"Error: No write access to {persist_directory}. Fixing permissions...")
            try:
                os.chmod(persist_directory, stat.S_IWUSR | stat.S_IRUSR | stat.S_IXUSR)
            except Exception as e:
                print(f"Failed to change directory permissions: {e}")
                return None

    try:
        # Load or create Chroma vector store
        if not os.listdir(persist_directory):  # Empty directory means no existing DB
            print("Initializing new ChromaDB instance...")
            db = Chroma.from_documents(documents, embedding_model, persist_directory=persist_directory)
            db.persist()
        else:
            print("Loading existing ChromaDB instance...")
            db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

        # For debugging: perform a similarity search and print the top result
        docs = db.similarity_search(user_query, k=1)
        print("\nRetrieved Document (for debugging):\n")
        print(format_docs(docs))
        return db.as_retriever()

    except Exception as e:
        print(f"Error while loading ChromaDB: {e}")
        return None


def generate_response(retriever, query):
    retrieved_chunks = retriever.get_relevant_documents(query)
    if not retrieved_chunks:
        return "No relevant document found."

    # Retrieve the top (most relevant) chunk and extract doc_id
    top_chunk = retrieved_chunks[0]
    doc_id = top_chunk.metadata.get("doc_id")
    chunk_index = top_chunk.metadata.get("chunk_index")

    # Find all chunks from the same document (using the global 'documents' variable)
    same_doc_chunks = [doc for doc in documents if doc.metadata.get("doc_id") == doc_id]
    same_doc_chunks = sorted(same_doc_chunks, key=lambda d: d.metadata.get("chunk_index", 0))

    # Define a window: e.g. 15 chunks before and after the top chunk
    start = max(0, chunk_index - 15)
    end = min(len(same_doc_chunks), chunk_index + 15)
    aggregated_context = "\n\n".join([doc.page_content for doc in same_doc_chunks[start:end]])

    # Build the chain and invoke it with the additional 'doc_name' variable
    chain = chat_prompt_template | model | StrOutputParser()
    input_vars = {"context": aggregated_context, "question": query, "doc_name": doc_id}
    return chain.invoke(input_vars)



def query(query_text , documents_local):
    
    retriever_local = load_embeddings(documents_local, query_text)
    response = generate_response(retriever_local, query_text)
    return response


In [4]:
# Global documents for dynamic neighbor retrieval
documents = load_documents()

In [44]:
query_text = """تفسیر بیت زیر چیست: /"
"یاد من کن پیش تخت آن عزیز /
تا مرا هم واخرد زین حبس نیز"""

In [45]:
retriever = load_embeddings(documents, query_text)

Loading existing ChromaDB instance...

Retrieved Document (for debugging):

یاد من کن پیش تخت آن عزیز. تا مرا هم واخرد زین حبس نیز. و مولوی اینجا را اشاره می کنیم. کی دهد


In [47]:
response = generate_response(retriever, query_text)

print(response)


بیت زیر از مثنوی معنوی مولانا مولوی است که در آن یوسف در زندان به همراه دیگر زندانیان به شاهزاده اش اشاره می‌کند. یوسف، پس از آزادی از زندان و رسیدن به مقام بالاتر، خواستار یاد آوری خود نزد شاهزاده شده و از او درخواست می‌کند که او را نیز از زندان آزاد کند. 

این بیت نشان‌دهنده تواضع و فروتنی یوسف در مقام بالاتر است و او خواهان آزادی دیگر زندانیان نیز می‌شود، به‌طوری که از این حبس و زندانی بیرون آیند. این بیت از زمینه‌های تواضع، انسانیت، و اهمیت یادآوری کردن به یکدیگر و کمک به دیگران برای رسیدن به آزادی و سعادت بیان می‌کند.

در جلسه شماره ۷۶ از تفسیر مثنوی معنوی عبدالکریم سروش، در مورد این بیت و معنای آن بحث شده است. این بیت در تفسیر اشعار مجاور و داستان مرتبط با آن در این جلسه به تفصیل بررسی شده است.
